# Netwerken week 3

## 3 Wat informatie over de netwerkinterface

1. Voer het commando `ifconfig` uit. 

```console
$ ifconfig
lo0: flags=8049<UP,LOOPBACK,RUNNING,MULTICAST> mtu 32768
        index 3 priority 0 llprio 3
        groups: lo
        inet6 ::1 prefixlen 128
        inet6 fe80::1%lo0 prefixlen 64 scopeid 0x3
        inet 127.0.0.1 netmask 0xff000000
em0: flags=808843<UP,BROADCAST,RUNNING,SIMPLEX,MULTICAST,AUTOCONF4> mtu 1500
        lladdr 08:00:27:b7:b9:f7
        index 1 priority 0 llprio 3
        groups: egress
        media: Ethernet autoselect (1000baseT full-duplex)
        status: active
        inet 192.168.2.26 netmask 0xffffff00 broadcast 192.168.2.255
enc0: flags=0<>
        index 2 priority 0 llprio 3
        groups: enc
        status: active
pflog0: flags=141<UP,RUNNING,PROMISC> mtu 33136
        index 4 priority 0 llprio 3
        groups: pflog
```

2. Welke interfaces hebben een IP-adres?
    - De volgende interfaces hebben een IP-adres
        1. lo0, het loopback interface. Deze heeft het localhost IP-adres `127.0.0.1`
        2. em0, de emulated ethernet port. Deze heeft het IP-adres `192.168.2.26` binnen mijn lokale netwerk
3. Geef voor iedere interface met een IP-adres het IP-blok in CIDR-notatie.
    1. `127.0.0.1/8` - De host is `127.0.0.1` waarbij de range `127.0.0.0` t/m `127.255.255.255` allemaal localhost zijn 
    2. `192.168.2.26/32` - De host is `192.168.2.26` en heeft maar 1 IP adres
4. Bepaal voor iedere interface het laagste en hoogste IP-adres binnen het netwerk. Geef aan hoe je deze hebt bepaald.
    1. `127.255.255.255` - Dit is het broadcast adres voor `127.255.255.0/24`. Dit is het laatste IP in de range `127.0.0.0` t/m `127.255.255.255`.
    2. `192.168.2.26/32` - Dit is het adres wat deze machine heeft gekregen van de lokale DHCP server. Dit is er maar 1 en dus is het host IP ook gelijk het hoogste ip binnen de range.
5. De netwerkinterface waarvan je de naam bij het opstarten van de virtuele machine hebt genoteerd, heeft ook een MAC-adres. Dit wordt met lladdr aangeduid in de uitvoer van ifconfig. De eerste drie bytes van het MAC-adres geven de fabrikant (Engels: ven- dor) van de netwerkadapter aan. Bepaal hiermee de fabrikant van deze netwerkadapter.
    - De eerste drie bytes van het MAC-adres van mijn netwerkadapter zijn `08 00 27`, dit hoort bij de vendor 'PCS Systemtechnik GmbH'
6. Met het commando route show krijg je een overzicht van de forwarding table van de virtuele machine. Gebruik de uitvoer van dit commando om het IP-adres van de default gateway te vinden.
    - De gateway is `192.168.2.255`

```console
$ route show
Routing tables

Internet:
Destination        Gateway            Flags   Refs      Use   Mtu  Prio Iface
default            modem              UGS        5        8     -     8 em0  
base-address.mcast localhost          URS        0        0 32768     8 lo0  
127/8              localhost          UGRS       0        0 32768     8 lo0  
localhost          localhost          UHhl       1        2 32768     1 lo0  
192.168.2/24       net                UCn        2     7562     -     4 em0  
net                08:00:27:b7:b9:f7  UHLl       0      111     -     1 em0  
server             xx:xx:xx:xx:xx:xx  UHLc       1     5389     -     3 em0  
modem              xx:xx:xx:xx:xx:xx  UHLch      1      159     -     3 em0  
192.168.2.255      net                UHb        0      105     -     1 em0  

Internet6:
...
```

7. Naast het IP-adres van de gateway wordt voor iedere route ook de interface waarop de gateway te bereiken is bijgehouden. Waarom?
    - Dit gebeurt omdat je via meerdere verschillende interfaces kan communiceren, zo kan je met lo0 communiceren naar je localhost en bestaat er soms ook een wireless adapter naast een ethernet port.
8. Om een IP-pakket naar de default gateway te sturen, dient dit ingekapseld te worden in een Ethernetframe. Hiervoor moet de machine het MAC- adres van deze gateway achterhalen met behulp van het Address Resolution Protocol.
Zodra het MAC-adres bekend is, zal de machine deze een tijdje bewaren in zijn ARP cache. Deze kun je bestuderen met het arp-commando. Voer dit commando uit met de optie -a. Wat zie je in deze uitvoer? Wat is het MAC-adres van de gateway?
    - Het MAC-adres van de gateway is `xx:xx:xx:5e:14:67`

```console
$ arp -a
Host                                 Ethernet Address    Netif Expire    Flags
net                                  08:00:27:b7:b9:f7     em0 permanent l
server                               xx:xx:xx:xx:xx:xx     em0 16m50s    
modem                                xx:xx:xx:5e:14:67     em0 19m21s    
```

## 4 Het Internet: een netwerk van netwerken

1. Ga de route naar `1.1.1.1` na met behulp van traceroute. Roep het programma hiertoe als volgt aan: `traceroute 1.1.1.1`

```console
$ traceroute 1.1.1.1
traceroute to 1.1.1.1 (1.1.1.1), 64 hops max, 40 byte packets
 1  modem (192.168.2.254)  0.668 ms  0.793 ms  1.296 ms
 2  195-190-228-137.fixed.kpn.net (195.190.228.137)  4.341 ms  2.499 ms  7.433 ms
 3  * * *
 4  * * *
 5  172.70.240.5 (172.70.240.5)  11.878 ms one.one.one.one (1.1.1.1)  5.15 ms  5.357 ms
 ```

2. Hoeveel knopen bevinden zich tussen jouw host en de bestemming?
    - Er zitten 5 knopen tussen mijn host en de bestemming.
3. De optie -A instrueert traceroute om de nummers van de autonome systemen in de uitvoer mee te nemen. Trek de route naar `1.1.1.1` nogmaals na, maar geef nu de -A optie mee aan traceroute.

```console
$ traceroute -A 1.1.1.1 
traceroute to 1.1.1.1 (1.1.1.1), 64 hops max, 40 byte packets
 1  modem (192.168.2.254)  1.032 ms  0.375 ms  0.61 ms
 2  195-190-228-137.fixed.kpn.net (195.190.228.137)  2.119 ms  1.92 ms  2.305 ms
 3  * * *
 4  * * *
 5  162.158.84.55 (162.158.84.55) [AS13335]  12.025 ms 172.70.240.3 (172.70.240.3) [AS13335]  25.187 ms one.one.one.one (1.1.1.1) [AS13335]  5.622 ms
 ```

4. Noteer de autonome systemen die op deze route liggen.
    1. AS13335
5. Een aantal IP-adressen behoort niet tot een AS. Verklaar waarom.
    - Niet ieder IP lijdt naar een deelnetwerk.
6. Voer volgend commando uit om te oefenen met het gebruik van whois. Welk bedrijf beheert volgend AS?
    - Dit AS wordt beheerd door YouTube / Google

```console
$ whois -a AS36561

#
# ARIN WHOIS data and services are subject to the Terms of Use
# available at: https://www.arin.net/resources/registry/whois/tou/
#
# If you see inaccuracies in the results, please report at
# https://www.arin.net/resources/registry/whois/inaccuracy_reporting/
#
# Copyright 1997-2022, American Registry for Internet Numbers, Ltd.
#

ASNumber:       36561
ASName:         YOUTUBE
ASHandle:       AS36561
RegDate:        2006-01-31
Updated:        2021-10-04
Ref:            https://rdap.arin.net/registry/autnum/36561

...
```

7. Bepaal nu de autonome systemen die jouw IP-pakketten doorlopen om bij 1.1.1.1 aan te komen.

```console
$ traceroute -A 1.1.1.1 | grep AS
traceroute to 1.1.1.1 (1.1.1.1), 64 hops max, 40 byte packets
 5  one.one.one.one (1.1.1.1) [AS13335]  5.276 ms 172.70.244.3 (172.70.244.3) [AS13335]  11.912 ms 172.70.240.5 (172.70.240.5) [AS13335]  22.205  
$ whois -a AS13335                

#
# ARIN WHOIS data and services are subject to the Terms of Use
# available at: https://www.arin.net/resources/registry/whois/tou/
#
# If you see inaccuracies in the results, please report at
# https://www.arin.net/resources/registry/whois/inaccuracy_reporting/
#
# Copyright 1997-2022, American Registry for Internet Numbers, Ltd.
#

ASNumber:       13335
ASName:         CLOUDFLARENET
ASHandle:       AS13335
RegDate:        2010-07-14
Updated:        2017-02-17
Comment:        All Cloudflare abuse reporting can be done via https://www.cloudflare.com/abuse
Ref:            https://rdap.arin.net/registry/autnum/13335
```

## 5 Netwerkverkeer analyseren met tcpdump
### 5.1 Een eerste poging

1. Type nu onderstaande opdracht in: `$ doas tcpdump`

```console
$ doas tcpdump
tcpdump: listening on em0, link-type EN10MB
...
223 packets received by filter
0 packets dropped by kernel
```

3. Wat valt je op aan de uitvoer?
    - Er is een hoop communicatie tussen de VM en host machine, maar weinig daarbuiten.

# 6 Filteren

1. Voer onderstaand commando in. Nu zal tcpdump verkeer van of naar poort 22 negeren. Daarnaast voorkomt de optie -n dat tcpdump getallen converteert naar namen. `$ doas tcpdump -n not port 22`

```console
$ doas tcpdump -n not port 22 > /tmp/tcplog.log
tcpdump: listening on em0, link-type EN10MB
```

2.  Laat tcpdump minimaal één minuut draaien. Maak een lijst van alle
poortnummers kleiner dan 1024 die je voorbij ziet komen.
